In [1]:
import nltk
import numpy as np
import urllib.request
import random
import string
import pyttsx3
import speech_recognition as sr

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords 
 

### Procesamiento de Voz

In [2]:
r = sr.Recognizer()
mic = sr.Microphone()
# sr.Microphone.list_microphone_names()

### Sintesis de Voz

In [3]:
engine = pyttsx3.init()
engine.setProperty('rate',150)
engine.setProperty('voice','spanish')

In [4]:
def speak(texto):
    engine.say(texto)
    engine.runAndWait()

In [5]:
#obtener el contenido HTML de la pagina web 
res = urllib.request.urlopen("https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/que-es-el-web-scraping/")
html = res.read()
soup = BeautifulSoup(html,"html.parser")

In [6]:
#observamos la etiquetas HTML
setTag = {x.name for x in soup.find_all(True)}
print(setTag)

{'p', 'body', 'main', 'div', 'img', 'ol', 'span', 'picture', 'noscript', 'h2', 'header', 'footer', 'strong', 'h1', 'em', 'h4', 'source', 'link', 'label', 'ul', 'article', 'style', 'html', 'i', 'br', 'li', 'time', 'button', 'head', 'title', 'form', 'input', 'section', 'nav', 'meta', 'base', 'script', 'a', 'fieldset', 'h3'}


In [7]:
def searchByTag(tagName):
    Content = ''
    for tag in soup.find_all(tagName):
        if(len(tag.text) > 144):
            Content += tag.text+'\n\n'
    return Content


In [8]:
Content = searchByTag('p')
print(Content)

Los motores de búsqueda, como Google, utilizan desde hace tiempo los denominados rastreadores web o crawlers, que exploran Internet en busca de términos definidos por el usuario. Los rastreadores son tipos especiales de bots, que visitan una página web tras otra para generar asociaciones con términos de búsqueda y categorizarlos. El primer rastreador web se creó ya en 1993, cuando se presentó el primer motor de búsqueda: Jumpstation.

Entre estas técnicas de rastreo se incluye el web scraping o webharvesting. Te explicamos cómo funciona, para qué se utiliza y cómo se puede bloquear en caso necesario.

Durante el web scraping (del inglés scraping = arañar/raspar) se extraen y almacenan datos de páginas web para analizarlos o utilizarlos en otra parte. Por medio de este raspado web se almacenan diversos tipos de información: por ejemplo, datos de contacto, tales como direcciones de correo electrónico o números de teléfono, o también términos de búsqueda o URL. Estos se almacenan en bases

In [9]:
raw = Content.lower()

In [10]:
sent_tokens = nltk.sent_tokenize(raw) # convertir corpus a una lista de sentencias (tekenizar en sentencias)
word_tokens = nltk.word_tokenize(raw) # convertir corpus a una lista de palabras (tokenizar en palabras)

lemmer = nltk.stem.WordNetLemmatizer()#

In [13]:
# word_tokens

In [14]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation) #remover signos de puntuacion (utilizamos la )

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] # Lematizar palabras tokenizadas

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [15]:
#Determinar la similutud del texto inserdado y el corpus

def response(user_response):# funcion respuesta (entrada: mensje de usuario)
    chatbot_response = '' #definir la respuesta del chatbot
    sent_tokens.append(user_response)# al listado de sentencias del corpus añadir al final de la lista el mensaje del usuario
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize,stop_words=stopwords.words('spanish'))# Lematizar y eliminar palbras
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # print("a",tfidf[-1])
    # print("------------------")
    # print(tfidf)
    # 3 Evaluar similitud coseno entre mensaje usuario (tfid[-1]) y el corpus(tfidf)
    vals = cosine_similarity(tfidf[-1],tfidf)
   

    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    
    flat.sort()
  
    req_tfidf = flat[-2]
    # print("------------------")
    # print(req_tfidf)
    # print(idx)
    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "Lo siento, no te he entendido"
        return chatbot_response
    
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response

In [16]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos dias")
SALUDOS_OUTPUTS = ["hola","hola, que tal?","Hola, Como te puedo ayudar?","hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        # print(word.lower() in SALUDOS_INPUTS)
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

In [15]:
# print(Content)

In [17]:
def speakAgent(msg):
    print(msg)
    speak(msg)


In [20]:
def chatBots():
    flag = True
    mode = True
    chat = True
    speakAgent("MI NOMBRE ES ROXANNE")
    while(mode == True):
        
        speakAgent("Si deseas hacerme preguntas por voz, por favor escriba, si, de lo contrario te respondere a las preguntas que me escribes en el formulario")
        res = input()
        
        if res.lower() in ["si","ok","y"]:
            print(res)
            chat = False
            mode = False
        elif res.lower() in ["no","not"]:
            mode = False

    while(flag == True):

        if chat:
            speakAgent("Por favor, escribame la pregunta?")
            user_response = input()
            user_response = user_response.lower()
        else:
            speak("Dime cual es tu duda")
            with sr.Microphone() as source:
                r.adjust_for_ambient_noise(source)
                audio = r.listen(source)
                
            text = r.recognize_google(audio, language='es',show_all=True)
            try:
                entrada = text['alternative'][0]['transcript']
                # print(entrada)
            except:
                entrada = ' '
                print("user:> ",entrada)

            user_response = entrada.lower()

        if(user_response not in ["salir","chao","adios","bay","exit"]):
            if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
                flag = True
                speakAgent(thanks_res)
            else:
                if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                    greets = saludos(user_response) 
                    speakAgent(greets)

                else:#si la palabra insertada no es un saludo --> Corpus
                    respond = response(user_response)
                    speakAgent(respond)
                    sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
        else:
            flag = False
            speakAgent("Nos vemos pronto, !Gracias por su visita¡") 


In [21]:
chatBots()

MI NOMBRE ES ROXANNE
Si deseas hacerme preguntas por voz, por favor escriba, si, de lo contrario te respondere a las preguntas que me escribes en el formulario
si
google es un buen ejemplo de web scraping.
Lo siento, no te he entendido
Lo siento, no te he entendido
en primer lugar, los scrapers deben tener en cuenta los derechos de propiedad intelectual de los sitios web.


KeyboardInterrupt: 